In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

In [2]:
model="gemini-2.0-flash"
emb_model = "models/gemini-embedding-001"

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

llm = ChatGoogleGenerativeAI(model=model)
embeddings = GoogleGenerativeAIEmbeddings(model=emb_model)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\sharath\Github\Projects\Loan prediction\loan-pred\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shash\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [126]:
embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [68]:
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [69]:
from langchain.document_loaders import JSONLoader
loader = JSONLoader(file_path="./synthetic_car_inventory_with_features_offers.json", jq_schema=".[]", text_content=False)
docs = loader.load()

In [127]:
vectorstore = FAISS.from_documents(docs, embeddings)

In [44]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5}),
    return_source_documents=True
)
qa_chain

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000023C89C77EF0>, default_metadata=()), 

In [48]:
user_input = "I want a car under $25000"
response = qa_chain.invoke(user_input)
print(response)

{'query': 'I want a car under $25000', 'result': 'I am sorry, I cannot fulfill this request. All the cars in my database are above $32000.\n', 'source_documents': [Document(id='7b31b289-a4c6-421b-a04a-50843d9ba5cf', metadata={'source': 'C:\\sharath\\Github\\Projects\\Loan prediction\\loan-pred\\assessments\\Pinnacle Intelligence\\codes\\simulator\\synthetic_car_inventory_with_features_offers.json', 'seq_num': 429}, page_content='{"id": "ACUIN429", "make": "Acura", "model": "Integra", "year": 2025, "trim": "Type S", "price": "58028$", "msrp": "59028$", "discount": "1000$", "mileage": 31209, "condition": "used", "fuel_type": "Diesel", "transmission": "CVT", "body_style": "SUV", "dealer": "Concord Acura", "location": "Aprilview, IA", "vin": "F1C7J7W6W2P1V9A6P2S2X6W9G3X6R0", "stock_number": "CONC429", "image_url": "https://example.com/images/acura_integra.jpg", "features": ["Auto Dimming Mirror", "Power Tailgate/Liftgate", "Remote Start", "Lane Departure Warning"], "offers": [{"name": "Pur

In [49]:
response.keys()

dict_keys(['query', 'result', 'source_documents'])

In [50]:
response['result']

'I am sorry, I cannot fulfill this request. All the cars in my database are above $32000.\n'

In [51]:
response['source_documents'][0].page_content


'{"id": "ACUIN429", "make": "Acura", "model": "Integra", "year": 2025, "trim": "Type S", "price": "58028$", "msrp": "59028$", "discount": "1000$", "mileage": 31209, "condition": "used", "fuel_type": "Diesel", "transmission": "CVT", "body_style": "SUV", "dealer": "Concord Acura", "location": "Aprilview, IA", "vin": "F1C7J7W6W2P1V9A6P2S2X6W9G3X6R0", "stock_number": "CONC429", "image_url": "https://example.com/images/acura_integra.jpg", "features": ["Auto Dimming Mirror", "Power Tailgate/Liftgate", "Remote Start", "Lane Departure Warning"], "offers": [{"name": "Purchase Allowance for Current Eligible Non-GM Owners and Lessees", "amount": -2000}]}'

In [52]:
user_input = "I want a car under $25K but don’t know what brand and let me know the discount and final price"
response = qa_chain.invoke(user_input)
print(response)

{'query': 'I want a car under $25K but don’t know what brand and let me know the discount and final price', 'result': 'I am sorry, I cannot fulfill that request. All of the Acura Integra cars in the provided data are above $50K.', 'source_documents': [Document(id='03ef7fb1-f0c5-494d-bcd0-d1441a2da139', metadata={'source': 'C:\\sharath\\Github\\Projects\\Loan prediction\\loan-pred\\assessments\\Pinnacle Intelligence\\codes\\simulator\\synthetic_car_inventory_with_features_offers.json', 'seq_num': 71}, page_content='{"id": "ACUIN071", "make": "Acura", "model": "Integra", "year": 2024, "trim": "Type S", "price": "66284$", "msrp": "66784$", "discount": "500$", "mileage": 49129, "condition": "certified", "fuel_type": "Hybrid", "transmission": "CVT", "body_style": "Coupe", "dealer": "Bay Area Honda", "location": "Victorhaven, AK", "vin": "M1D1E1D2K7S7P7W8D5E6Z5P3U4K5C2", "stock_number": "BAY071", "image_url": "https://example.com/images/acura_integra.jpg", "features": ["Lane Departure Warnin

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = """
Given Knowledge Base and User Query understand the query and answer it appropriately.
User Query: {user_input}
"""
prompt = PromptTemplate(
    input_variables=["user_input"],
    template=prompt_template
)
prompt

PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='\nGiven Knowledge Base and User Query understand the query and answer it appropriately.\nUser Query: {user_input}\n')

In [75]:
from langchain.memory import (
    ChatMessageHistory,
    ConversationBufferMemory,
    ConversationSummaryMemory,
    ConversationSummaryBufferMemory,
    ConversationBufferWindowMemory,
    ConversationTokenBufferMemory,
    ConversationKGMemory,
    ConversationEntityMemory,
    ConversationStringBufferMemory,
    ConversationVectorStoreTokenBufferMemory
)
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.conversational_retrieval.prompts import QA_PROMPT

In [56]:
QA_PROMPT

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:")

In [ ]:
chain = prompt | llm

chain.invoke(user_input)

In [78]:
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
question_generator_chain

LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='\nGiven Knowledge Base and User Query understand the query and answer it appropriately.\nUser Query: {user_input}\n'), llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000023C89C77EF0>, default_metadata=()), output_parser=StrOutputParser(), llm_kwargs={})

In [82]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
message_history = ChatMessageHistory()
chat_memory = ConversationBufferMemory(
    memory_key="chat_history",
    output_key="answer",
    chat_memory=message_history,
    return_messages=True
)


In [84]:
chain = ConversationalRetrievalChain(
    # question_generator=question_generator_chain,
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    memory=chat_memory,
    get_chat_history=lambda h: h,
    # return_source_document=True
)

ValidationError: 4 validation errors for ConversationalRetrievalChain
combine_docs_chain
  Field required [type=missing, input_value={'llm': ChatGoogleGenerat... at 0x0000023E2D7EF7E0>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
question_generator
  Field required [type=missing, input_value={'llm': ChatGoogleGenerat... at 0x0000023E2D7EF7E0>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
llm
  Extra inputs are not permitted [type=extra_forbidden, input_value=ChatGoogleGenerativeAI(mo...0>, default_metadata=()), input_type=ChatGoogleGenerativeAI]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden
chain_type
  Extra inputs are not permitted [type=extra_forbidden, input_value='stuff', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [85]:
from langchain.chains import create_retrieval_chain, create_history_aware_retriever

In [86]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

In [ ]:
condensed_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", )
    ]
)

In [73]:
chain.invoke(user_input)

{'question': 'I want a car under $25K but don’t know what brand and let me know the discount and final price',
 'chat_history': [HumanMessage(content='I want a car under $25K but don’t know what brand and let me know the discount and final price', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I am sorry, I cannot fulfill this request. The cars in the provided data are above $50K.', additional_kwargs={}, response_metadata={})],
 'answer': 'I am sorry, I cannot fulfill this request. The cars in the provided data are above $50K.'}

# Using Langgraph

In [88]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

In [108]:
from langchain_core.prompts import PromptTemplate

prompt_template = """
You are an expert AI car-buying assistant.

You will receive a knowledge base of vehicle listings in JSON format under {context}.
Each vehicle contains keys like:
- make, model, year, price, msrp, discount, fuel_type, body_style, condition, mileage, dealer

User question: {question}

Guidelines:
1. ONLY use information present in the context. Do not fabricate or guess.
2. If user asks about prices, always calculate the **final price = price - discount** (assume both are in USD).
3. If no vehicle matches the user's criteria, respond politely with: "Sorry, I couldn’t find any vehicles matching your criteria."
4. If vehicles match, show up to **2 best suggestions**, along with key details: make, model, year, final price, discount, and dealer.
5. Be concise, friendly, and factual.

Now based on the context and question, provide the best response:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)


In [109]:
print(prompt.template)


You are an expert AI car-buying assistant.

You will receive a knowledge base of vehicle listings in JSON format under {context}.
Each vehicle contains keys like:
- make, model, year, price, msrp, discount, fuel_type, body_style, condition, mileage, dealer

User question: {question}

Guidelines:
1. ONLY use information present in the context. Do not fabricate or guess.
2. If user asks about prices, always calculate the **final price = price - discount** (assume both are in USD).
3. If no vehicle matches the user's criteria, respond politely with: "Sorry, I couldn’t find any vehicles matching your criteria."
4. If vehicles match, show up to **2 best suggestions**, along with key details: make, model, year, final price, discount, and dealer.
5. Be concise, friendly, and factual.

Now based on the context and question, provide the best response:



In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
dense_retriever = vectorstore.as_retriever()

# === Step 3: Build BM25 Sparse Retriever ===
sparse_retriever = BM25Retriever.from_documents(docs)
sparse_retriever.k = 5  # top K matches

# === Step 4: Hybrid Retriever ===
hybrid_retriever = EnsembleRetriever(
    retrievers=[dense_retriever, sparse_retriever],
    weights=[0.6, 0.4]  # Tune weights as needed
)

In [ ]:
hybrid_retriever

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023C8650FD70>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x0000023C8099C0E0>, k=5)], weights=[0.6, 0.4])

In [121]:
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


def retrieve(state: State):
    # retrived_docs = vectorstore.similarity_search(state['question'])
    retrived_docs = hybrid_retriever.get_relevant_documents(state['question'])
    return {"context": retrived_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state['context'])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [122]:
response = graph.invoke({"question": user_input})
print(response['answer'])

C:\Users\shash\AppData\Local\Temp\ipykernel_23956\774762254.py:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrived_docs = hybrid_retriever.get_relevant_documents(state['question'])


Sorry, I couldn’t find any vehicles matching your criteria.



In [115]:
response['context']

[Document(id='04baa194-8138-4f3e-812b-c6981cd9ea01', metadata={'source': 'C:\\sharath\\Github\\Projects\\Loan prediction\\loan-pred\\assessments\\Pinnacle Intelligence\\codes\\simulator\\synthetic_car_inventory_with_features_offers.json', 'seq_num': 206}, page_content='{"id": "ACUIN206", "make": "Acura", "model": "Integra", "year": 2023, "trim": "Type S", "price": 84027, "msrp": 87027, "discount": 3000, "mileage": 25313, "condition": "certified", "fuel_type": "Electric", "transmission": "CVT", "body_style": "Sedan", "dealer": "Bay Area Honda", "location": "Mooreshire, DE", "vin": "H6T8K4C2U6U8D4X0B1V8T4H6A4B2V7", "stock_number": "BAY206", "image_url": "https://example.com/images/acura_integra.jpg", "features": ["Auto Dimming Mirror", "Cooled Seats", "4WD/AWD", "Remote Start", "Wi-Fi Hotspot", "Apple CarPlay", "Android Auto"], "offers": [{"name": "GM Educator Offer", "amount": -500}, {"name": "Purchase Allowance for Current Eligible Non-GM Owners and Lessees", "amount": -2000}, {"name":

In [17]:

from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://www.californiacaronline.com/searchnew.aspx"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[[Document(metadata={'source': 'https://www.californiacaronline.com/searchnew.aspx', 'title': 'Acura Vehicle Inventory - Concord Acura dealer in Dublin CA - New and Used Acura dealership Dublin Fremont Livermore CA', 'description': 'Search Acura Inventory at Dosanjh Family Auto Group for ', 'language': 'en'}, page_content='\n\n\n\n\nAcura Vehicle Inventory - Concord Acura dealer in Dublin CA - New and Used Acura dealership Dublin Fremont Livermore CA\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDosanjh Family Auto Group\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n4200 John Monego Court Dublin, CA 94568\n\n\n\n\n\n\n\n\nService\n\n\n\n\n\n\n\nMap\n\n\n\n\n\n\n\nContact\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDosanjh Family Auto Group\n\n\n\n\n\n\n\n\n\n Call \n\n Directions\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBuy Online\n\n\n\n\nBuy Online\n\n\n\n\nBuy New\n\n\n\n\nBuy Used\n\n\n\n\n\n\nNew Models\n\n\n\n\nAll 

In [18]:
docs[0][0].page_content.strip()[:1000]

'Acura Vehicle Inventory - Concord Acura dealer in Dublin CA - New and Used Acura dealership Dublin Fremont Livermore CA\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDosanjh Family Auto Group\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n4200 John Monego Court Dublin, CA 94568\n\n\n\n\n\n\n\n\nService\n\n\n\n\n\n\n\nMap\n\n\n\n\n\n\n\nContact\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDosanjh Family Auto Group\n\n\n\n\n\n\n\n\n\n Call \n\n Directions\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBuy Online\n\n\n\n\nBuy Online\n\n\n\n\nBuy New\n\n\n\n\nBuy Used\n\n\n\n\n\n\nNew Models\n\n\n\n\nAll New Vehicles\n\n\n\n\nShop Acura\n\n\n\n\nShop Buick\n\n\n\n\nShop Cadillac\n\n\n\n\nShop Chevrolet\n\n\n\n\nShop GMC\n\n\n\n\nShop INFINITI\n\n\n\n\nShop Kia\n\n\n\n\nShop Nissan\n\n\n\n\nShop Toyota\n\n\n\n\n\n\nPre-Owned Vehicles\n\n\n\n\nPre-Owned Vehicles\n\n\n\n\nSearch Inventory\n\n\n\n\nCertified Pre-Owned Vehicles\n\n\n\n\n

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

In [20]:
doc_splits[0].page_content.strip()

'Acura Vehicle Inventory - Concord Acura dealer in Dublin CA - New and Used Acura dealership Dublin Fremont Livermore CA\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDosanjh Family Auto Group\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n4200 John Monego Court Dublin, CA 94568\n\n\n\n\n\n\n\n\nService'

In [22]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits, embedding=embeddings
)
retriever = vectorstore.as_retriever()

In [27]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_information",
    "Search and return information about cars.",
)
result = retriever_tool.invoke({"query": "I want a car under $25K but don’t know what brand"})
result

'Shop Under $20K\n\n\n\n\n\n\nService\n\n\n\n\nSchedule Service\n\n\n\n\n\n\nFinance\n\n\n\n\nGet Financed\n\n\n\n\n\n\nAbout Us\n\n\n\n\nWhy Buy\n\n\n\n\nOur Locations\n\n\n\n\nCareers / Employment\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\nSearch Inventory\n\nShop Cadillac\n\n\n\n\nShop Chevrolet\n\n\n\n\nShop GMC\n\n\n\n\nShop INFINITI\n\n\n\n\nShop Kia\n\n\n\n\nShop Nissan\n\n\n\n\nShop Toyota\n\n\n\n\n\n\nPre-Owned Vehicles\n\n\n\n\nPre-Owned Vehicles\n\n\n\n\nSearch Inventory\n\n\n\n\nCertified Pre-Owned Vehicles\n\n\n\n\nShop Sedans\n\n\n\n\nShop SUVs\n\n\n\n\nShop Trucks\n\n\n\n\nShop Under $20K\n\nBuy Online\n\n\n\n\nBuy Online\n\n\n\n\nBuy New\n\n\n\n\nBuy Used\n\n\n\n\n\n\nNew Models\n\n\n\n\nAll New Vehicles\n\n\n\n\nShop Acura\n\n\n\n\nShop Buick\n\n\n\n\nShop Cadillac\n\n\n\n\nShop Chevrolet\n\n\n\n\nShop GMC\n\n\n\n\nShop INFINITI\n\n\n\n\nShop Kia\n\n\n\n\nShop Nissan\n\n\n\n\nShop Toyota\n\n\n\n\n\n\nPre-Owned Vehicles\n\nShop Buick\n\n\n\n\nShop Cadillac\n\n\n\n\nShop Chevrolet\

In [28]:
print(result)

Shop Under $20K






Service




Schedule Service






Finance




Get Financed






About Us




Why Buy




Our Locations




Careers / Employment









 





Search Inventory

Shop Cadillac




Shop Chevrolet




Shop GMC




Shop INFINITI




Shop Kia




Shop Nissan




Shop Toyota






Pre-Owned Vehicles




Pre-Owned Vehicles




Search Inventory




Certified Pre-Owned Vehicles




Shop Sedans




Shop SUVs




Shop Trucks




Shop Under $20K

Buy Online




Buy Online




Buy New




Buy Used






New Models




All New Vehicles




Shop Acura




Shop Buick




Shop Cadillac




Shop Chevrolet




Shop GMC




Shop INFINITI




Shop Kia




Shop Nissan




Shop Toyota






Pre-Owned Vehicles

Shop Buick




Shop Cadillac




Shop Chevrolet




Shop GMC




Shop INFINITI




Shop Kia




Shop Nissan




Shop Toyota






Pre-Owned Vehicles




Pre-Owned Vehicles




Search Inventory




Certified Pre-Owned Vehicles




Shop Sedans




Shop SUVs




Shop Trucks
